In [5]:
import os
import json
import base64
import requests
from typing import Optional




class ThucChienImageGenerator:
   """Client sinh hình ảnh qua gateway AI Thực Chiến (chuẩn OpenAI / Gemini)."""


   def __init__(self,
                base_url: str = "https://api.thucchien.ai/v1",
                api_key: Optional[str] = None,
                debug: bool = False):
       self.base_url = base_url.rstrip("/")
       self.api_key = api_key or os.getenv("LITELLM_API_KEY", "")
       self.debug = debug
       self.headers = {
           "Content-Type": "application/json",
           "Authorization": f"Bearer {self.api_key}"
       }


   def generate_image(self,
                      prompt: str,
                      model: str = "gemini-2.5-flash-image-preview",
                      output_path: str = "generated_image.png") -> Optional[str]:
       """
       Sinh ảnh từ mô tả văn bản (prompt) bằng model gemini-flash-image-preview.


       Args:
           prompt: mô tả hình ảnh cần sinh
           model: model dùng để sinh (vd: gemini-2.5-flash-image-preview)
           output_path: nơi lưu ảnh kết quả
       """
       url = f"{self.base_url}/chat/completions"
       data = {
           "model": model,
           "messages": [
               {"role": "user", "content": prompt}
           ]
       }


       print(f"🎨 Generating image using model: {model}")
       print(f"🖋️ Prompt: {prompt}\n")


       try:
           response = requests.post(url, headers=self.headers, data=json.dumps(data))
           response.raise_for_status()
           result = response.json()


           if self.debug:
               print("\n--- RAW RESPONSE ---")
               print(json.dumps(result, indent=2, ensure_ascii=False))
               print("--------------------\n")


           # 🧩 Trích xuất dữ liệu ảnh base64
           base64_string = result['choices'][0]['message']['images'][0]['image_url']['url']


           # Xử lý prefix 'data:image/png;base64,' nếu có
           if ',' in base64_string:
               _, encoded = base64_string.split(',', 1)
           else:
               encoded = base64_string


           image_data = base64.b64decode(encoded)


           # Lưu ảnh
           with open(output_path, 'wb') as f:
               f.write(image_data)


           print(f"✅ Image saved successfully: {output_path}")
           return output_path


       except requests.exceptions.RequestException as e:
           print(f"❌ Request failed: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None


       except (KeyError, IndexError) as e:
           print(f"⚠️ Failed to parse image data from response: {e}")
           if 'response' in locals():
               print(f"Response body: {response.text}")
           return None




# ====================== MAIN ======================
def main():
   base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/v1")
   api_key = os.getenv("LITELLM_API_KEY", "sk-JJytUYWGmr5BGv9rVurb2Q")


   generator = ThucChienImageGenerator(base_url=base_url, api_key=api_key, debug=False)


   prompt = (
       """Hãy tạo cho tôi một hình ảnh kích thước 10*7cm 
       Phần góc trên bên phải (phần nhỏ hơn phần góc dưới bên trái) là hình ảnh một gia đình (cha mẹ, con cái) đang nắm tay nhau vui vẻ đi chơi
       Phần góc dưới bên trái là hình ảnh trắng đen miêu tả một gia đình tan vỡ vì người chồng nghiện ngập
       Giữa hai hình ảnh là một vết xé rách chéo (như xé đôi tờ giấy ấy)
       High resolution, photorealistic, 8k."""
   )


   generator.generate_image(prompt, output_path="futuristic_city.png")




if __name__ == "__main__":
   main()


🎨 Generating image using model: gemini-2.5-flash-image-preview
🖋️ Prompt: Hãy tạo cho tôi một hình ảnh kích thước 10*7cm 
       Phần góc trên bên phải (phần nhỏ hơn phần góc dưới bên trái) là hình ảnh một gia đình (cha mẹ, con cái) đang nắm tay nhau vui vẻ đi chơi
       Phần góc dưới bên trái là hình ảnh trắng đen miêu tả một gia đình tan vỡ vì người chồng nghiện ngập
       Giữa hai hình ảnh là một vết xé rách chéo (như xé đôi tờ giấy ấy)
       High resolution, photorealistic, 8k.

⚠️ Failed to parse image data from response: list index out of range
Response body: {"id":"eM_5aPfdNdS2vr0PnfTP6QQ","created":1761202039,"model":"gemini-2.5-flash-image-preview","object":"chat.completion","choices":[{"finish_reason":"stop","index":0,"message":{"content":"Tuy nhiên, tôi không thể tạo ra hình ảnh với các kích thước cụ thể như bạn yêu cầu (10*7cm) hoặc theo định dạng \"phần góc trên bên phải (phần nhỏ hơn phần góc dưới bên trái)\". Tôi cũng không thể trực tiếp tạo ra hình ảnh trắng đen.\n\n